In [1]:
#@markdown --------------- 
#@markdown ## **⚠️❗ Ejecute esta celda para descargar Shoes dataset❗⚠️** 
#@markdown ### Esta celda creará la carpeta ```/content/genres_original```

!pip install -qq gdown
!gdown -qq "https://drive.google.com/u/1/uc?id=1o8RYJtI7FCdyPqbglgXf17bxajy8_CD1&export=download" -O /genres_original_short.zip
!mkdir /content/genres_original/
!unzip -qq /genres_original_short.zip -d /content/genres_original
!rm -r /genres_original_short.zip
print ("Done!")
#@markdown ---------------

Done!


In [2]:
import os
import librosa
import math
import json 
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
import tensorflow.keras as keras

from tqdm import tqdm
import time

In [3]:
dataset_path = "/content/genres_original"
json_path = r"data.json"
SAMPLE_RATE = 22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [21]:
# Parámetros de configuración
n_mfcc = 13
n_fft = 2048
hop_length = 512
num_segments = 10

# Data storage dictionary
data = {
    "mapping": [],
    "mfcc": [],
    "labels": [],
}

# Cálculo de variables auxiliares
samples_ps = int(SAMPLES_PER_TRACK / num_segments)  # ps = per segment
expected_vects_ps = math.ceil(samples_ps / hop_length)

# Recorrido de géneros musicales
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    # Se asegura de no estar en la raíz
    if dirpath != dataset_path:
        # Guardar la etiqueta semántica
        semantic_label = os.path.basename(dirpath)
        data["mapping"].append(semantic_label)
        print(f"Processing: {semantic_label}")

        # Procesar archivos para el género específico
        for f in tqdm(filenames, ncols=80):
            if f == "jazz.00054.wav":
                # Como librosa solo lee archivos <1Mb
                continue
            else:
                # Cargar el archivo de audio
                file_path = os.path.join(dirpath, f)
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)
                for s in range(num_segments):
                    start_sample = samples_ps * s
                    finish_sample = start_sample + samples_ps

                    # Obtener los coeficientes MFCC
                    mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_mfcc=n_mfcc,
                                                hop_length=hop_length,
                                                n_fft=n_fft)

                    mfcc = mfcc.T

                    # Guardar los coeficientes MFCC si tienen la longitud esperada
                    if len(mfcc) == expected_vects_ps:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1)

# Guardar los datos en formato JSON
with open(json_path, "w") as f:
    json.dump(data, f, indent=4)

Processing: rock


100%|█████████████████████████████████████████| 100/100 [00:15<00:00,  6.66it/s]


Processing: country


100%|█████████████████████████████████████████| 100/100 [00:13<00:00,  7.37it/s]


Processing: hiphop


...

100%|█████████████████████████████████████████| 100/100 [00:13<00:00,  7.21it/s]


Processing: reggae


100%|█████████████████████████████████████████| 100/100 [00:14<00:00,  7.06it/s]


Processing: classical


100%|█████████████████████████████████████████| 100/100 [00:18<00:00,  5.50it/s]


In [22]:
with open("data.json","r") as f:
    data = json.load(f)
    # Convert list to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
      
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [23]:
X_train.shape

(7988, 130, 13)

In [24]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = "relu", input_shape = (130, 13, 1)))
model.add(MaxPool2D((3, 3), strides=(2, 2), padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPool2D((3, 3), strides=(2, 2), padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(32, (2, 2), activation = "relu"))
model.add(MaxPool2D((2, 2), strides=(2, 2), padding="same"))
model.add(BatchNormalization())

model.add(Conv2D(16, (1, 1), activation = "relu"))
model.add(MaxPool2D((1, 1), strides=(2, 2), padding="same"))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(10, activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 11, 64)       640       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 6, 64)        0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 64, 6, 64)        256       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 62, 4, 32)         18464     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 31, 2, 32)        0         
 2D)                                                             
                                                      

In [25]:
adam = optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [26]:
hist = model.fit(X_train, y_train,\
                 epochs = 100 , batch_size = 200 , \
                    verbose = 1 , validation_split = 0.2)

Epoch 1/100
32/32 [==============================] - 5s 47ms/step - loss: 2.1717 - accuracy: 0.2798 - val_loss: 3.1409 - val_accuracy: 0.1733
Epoch 2/100
32/32 [==============================] - 0s 14ms/step - loss: 1.6141 - accuracy: 0.4230 - val_loss: 2.6488 - val_accuracy: 0.2040
Epoch 3/100
32/32 [==============================] - 0s 15ms/step - loss: 1.3885 - accuracy: 0.4989 - val_loss: 2.3443 - val_accuracy: 0.2403
Epoch 4/100
32/32 [==============================] - 0s 13ms/step - loss: 1.2464 - accuracy: 0.5446 - val_loss: 2.0609 - val_accuracy: 0.3004
Epoch 5/100
32/32 [==============================] - 0s 13ms/step - loss: 1.1354 - accuracy: 0.5881 - val_loss: 1.6363 - val_accuracy: 0.4374
Epoch 6/100
32/32 [==============================] - 0s 14ms/step - loss: 1.0347 - accuracy: 0.6283 - val_loss: 1.5057 - val_accuracy: 0.4656
Epoch 7/100
32/32 [==============================] - 0s 14ms/step - loss: 0.9692 - accuracy: 0.6599 - val_loss: 1.3425 - val_accuracy: 0.5382
Epoch 

In [27]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy}")

63/63 [==============================] - 0s 3ms/step - loss: 1.3599 - accuracy: 0.7618
Test accuracy: 0.7617617845535278
